In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_data = pd.read_csv('../input/pubg-finish-placement-prediction/train_V2.csv', index_col = 0)

In [ ]:
train_data.head(2)

In [ ]:
test_data = pd.read_csv('../input/pubg-finish-placement-prediction/test_V2.csv')

In [ ]:
test_data.head(2)

In [ ]:
import lightgbm as lgb
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=np.nan)

fit (подгонка) - вычисляет среднее и стандартное значение для последующего масштабирования.

transform (преобразование) - использует ранее вычисленное фитом (это ответ на вопрос почему. нам не надо пересчитывать для тестовой выборки, хотя бы потому что тренировочная больше). А вот все такие оценщики подгоняются по обучающим данным. Для тестового набора применяется прогнозирование на основе обучающего.

In [ ]:
train_data.info()

In [ ]:
train_data['matchType'][0:1]

In [ ]:
train_data['matchType'] = encoder.fit_transform(np.array(train_data['matchType']).reshape(-1, 1))

In [ ]:
test_data['matchType'] = encoder.transform(np.array(test_data['matchType']).reshape(-1, 1))

In [ ]:
train_data['matchType'][0:1]

In [ ]:
train_data.isnull().sum()

In [ ]:
[train_data[train_data['winPlacePerc'].isnull()].index.tolist()]

In [ ]:
train_data = train_data.drop(train_data[train_data['winPlacePerc'].isnull()].index.tolist(), axis=0)

In [ ]:
train_data.isnull().sum()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
y = np.array(train_data['winPlacePerc'])

In [ ]:
X = np.array(train_data.drop(['matchId', 'winPlacePerc'], axis=1))

In [ ]:
len(X)

Меня раздражал этот глупый признак matchId

In [ ]:
train = train_data.drop(['matchId'], axis=1)

In [ ]:
df_train, df_val = train_test_split(train, test_size=0.2, random_state=42, shuffle=True)

In [ ]:
my_idea = np.array(train_data.corr()[26:27])[0, 1:]

In [ ]:
len(my_idea)

In [ ]:
arr_my = sorted(my_idea, reverse=True)
my_columns = list(train_data.drop(['matchId', 'groupId'], axis=1).columns)

In [ ]:
len(my_columns)

In [ ]:
for i in arr_my:
    var = [j for j, e in enumerate(my_idea) if e == i] 
    var2 = my_columns[var[0]]
    print(var2)

In [ ]:
my_corr = train_data.corr()[26:27]

In [ ]:
my_corr['rideDistance']

In [ ]:
features = ['walkDistance', 'killPlace', 'boosts',
            'weaponsAcquired', 'damageDealt',
            'heals', 'kills', 'longestKill',
            'killStreaks', 'rideDistance']

label = 'winPlacePerc'


In [ ]:
dtrain = lgb.Dataset(data=df_train[features],
                     label=df_train[label],
                     feature_name=features)

dval = lgb.Dataset(data=df_val[features],
                   label=df_val[label],
                   feature_name=features)

In [ ]:
params = {
        'boosting_type': 'gbdt',
        'objective': 'mae',
        'metric': 'mae',
        'num_leaves': 40,
        'max_depth': None,
        'learning_rate': 0.1,
        'min_data_in_leaf': 10,
        'feature_fraction': 0.6,
        'bagging_fraction': 0.6,
        'bagging_freq': 1,
        #'bagging_freq': 5,
        'num_threads': 6,
    
        'cat_smooth': 10,
        'max_cat_threshold': 16,
        'max_cat_to_onehot': 4,
    }

In [ ]:
model = lgb.train(params, dtrain, 
              # указываем валидационный датасет и тренировочный (хотим посмотреть качество и на нем тоже)
              # однако он будет проигнорирован механикой ранней остановки
              valid_sets=(dtrain, dval),
              # поставим очень большое количество итераций бустинга
              num_boost_round=10000,
              # но будем использовать раннюю остановку по качеству на валидационной выборке
              early_stopping_rounds=25,
              #будем выводить промежуточные результаты каждые 25 итераций
              verbose_eval=25)

**[1482]	training's l1: 0.0715554	valid_1's l1: 0.0718205**

In [ ]:
params2 = {
        'boosting_type': 'goss',
        'objective': 'mae',
        'metric': 'mae',
        'num_leaves': 70,
        'max_depth': None,
        'learning_rate': 0.1,
        'min_data_in_leaf': 15,
        'feature_fraction': 0.5,
      #  'bagging_fraction': 0.5,
      # 'bagging_freq': 1,
        #'bagging_freq': 5,
        'num_threads': 6
    
      #  'cat_smooth': 10,
      #  'max_cat_threshold': 16,
      #  'max_cat_to_onehot': 4,
    }

In [ ]:
model = lgb.train(params2, dtrain, 
              # указываем валидационный датасет и тренировочный (хотим посмотреть качество и на нем тоже)
              # однако он будет проигнорирован механикой ранней остановки
              valid_sets=(dtrain, dval),
              # поставим очень большое количество итераций бустинга
              num_boost_round=10000,
              # но будем использовать раннюю остановку по качеству на валидационной выборке
              early_stopping_rounds=25,
              #будем выводить промежуточные результаты каждые 25 итераций
              verbose_eval=25)

**[934]	training's l1: 0.0713204	valid_1's l1: 0.0717142**

In [ ]:
features = ['walkDistance', 'killPlace', 'boosts', 'weaponsAcquired', 'damageDealt', 'heals', 'kills', 'longestKill',
            'killStreaks', 'rideDistance', 'assists', 'DBNOs', 'headshotKills', 'revives', 'swimDistance',
            'vehicleDestroys', 'numGroups', 'maxPlace', 'roadKills', 'matchType', 'teamKills', 'rankPoints',
            'killPoints', 'winPoints', 'matchDuration']

label = 'winPlacePerc'

In [ ]:
dtrain = lgb.Dataset(data=df_train[features],
                     label=df_train[label],
                     feature_name=features)

dval = lgb.Dataset(data=df_val[features],
                   label=df_val[label],
                   feature_name=features)


In [ ]:
params2 = {
        'boosting_type': 'goss',
        'objective': 'mae',
        'metric': 'mae',
        'num_leaves': 70,
        'max_depth': None,
        'learning_rate': 0.1,
        'min_data_in_leaf': 15,
        'feature_fraction': 0.5,
      #  'bagging_fraction': 0.5,
      # 'bagging_freq': 1,
        #'bagging_freq': 5,
        'num_threads': 6
    
      #  'cat_smooth': 10,
      #  'max_cat_threshold': 16,
      #  'max_cat_to_onehot': 4,
    }

In [ ]:
model = lgb.train(params2, dtrain, 
              # указываем валидационный датасет и тренировочный (хотим посмотреть качество и на нем тоже)
              # однако он будет проигнорирован механикой ранней остановки
              valid_sets=(dtrain, dval),
              # поставим очень большое количество итераций бустинга
              num_boost_round=10000,
              # но будем использовать раннюю остановку по качеству на валидационной выборке
              early_stopping_rounds=25,
              #будем выводить промежуточные результаты каждые 25 итераций
              verbose_eval=25)

**[2448]	training's l1: 0.0546092	valid_1's l1: 0.0556186**

In [ ]:
features = ['walkDistance', 'killPlace', 'boosts', 'weaponsAcquired', 'damageDealt', 'heals', 'kills', 'longestKill',
            'killStreaks', 'rideDistance', 'assists', 'DBNOs', 'headshotKills', 'revives', 'swimDistance',
            'vehicleDestroys', 'numGroups', 'maxPlace', 'roadKills', 'matchType', 'teamKills', 'rankPoints',
            'killPoints', 'winPoints', 'matchDuration']

categorical_feature = ['matchType']

label = 'winPlacePerc'

In [ ]:
dtrain = lgb.Dataset(data=df_train[features],
                     label=df_train[label],
                     feature_name=features,
                     categorical_feature=categorical_feature)

dval = lgb.Dataset(data=df_val[features],
                   label=df_val[label],
                   feature_name=features,
                   categorical_feature=categorical_feature)

In [ ]:
params2 = {
        'boosting_type': 'goss',
        'objective': 'mae',
        'metric': 'mae',
        'num_leaves': 70,
        'max_depth': None,
        'learning_rate': 0.1,
        'min_data_in_leaf': 15,
        'feature_fraction': 0.5,
      #  'bagging_fraction': 0.5,
      # 'bagging_freq': 1,
        #'bagging_freq': 5,
        'num_threads': 6
    
      #  'cat_smooth': 10,
      #  'max_cat_threshold': 16,
      #  'max_cat_to_onehot': 4,
    }

In [ ]:
model = lgb.train(params2, dtrain, 
              # указываем валидационный датасет и тренировочный (хотим посмотреть качество и на нем тоже)
              # однако он будет проигнорирован механикой ранней остановки
              valid_sets=(dtrain, dval),
              # поставим очень большое количество итераций бустинга
              num_boost_round=10000,
              # но будем использовать раннюю остановку по качеству на валидационной выборке
              early_stopping_rounds=25,
              #будем выводить промежуточные результаты каждые 25 итераций
              verbose_eval=25)

**[2505]	training's l1: 0.0545455	valid_1's l1: 0.0555608**

**Задание 2.2**

In [ ]:
from sklearn.model_selection import StratifiedKFold

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
def eval_lgb(params):
    base_params = {
        'boosting_type': 'gbdt',
        'objective': 'mae',
        'metric': 'mae',
        'num_threads': 6
    }
    folds = KFold(5, shuffle = True, random_state=42)

    base_params.update(params)
    results = lgb.cv(
        params,
        dtrain,
        num_boost_round = 10000,
        folds = folds.split(df_train[features]),
        early_stopping_rounds = 25,
        metrics='mae'
        #stratified = KFold(n_splits=10, shuffle=True, random_state=42)
        
    )
    
    return results

In [ ]:
res = eval_lgb(params)